In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."
assert TAVILY_API_KEY, "TAVILY_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")
print("Tavily Key loaded:", TAVILY_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)

# Example: Use Gemini
chain = prompt | gemini_llm
response = chain.invoke({"product": "smart home devices"})
print("Gemini response:", response)

# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
Tavily Key loaded: tvly-d...
Gemini response: content='Okay, let\'s brainstorm some creative names for a smart home device company, playing on different aspects like intelligence, comfort, connectivity, and the future.\n\nHere are a few categories and names within them:\n\n---\n\n### I. Emphasizing Intelligence & Automation\n\n1.  **Cognito Home:** (From "cognition," implying thinking and knowing)\n2.  **AuraLogic:** (A subtle intelligence that surrounds your home)\n3.  **Synapse Smart:** (Referring to the connections in a brain, for interconnected devices)\n4.  **Verve AI:** (Liveliness and intelligent automation)\n5.  **Nucleus Smart:** (The core intelligence of your home)\n6.  **Sentient Living:** (Suggesting the home itself has awareness)\n\n---\n\n### II. Focusing on Comfort, Ease & Living\n\n7.  **HavenFlow:** (A place of peace where everything flows seamlessly)\n8.  **Loomia:** (Suggests weaving technology into the fabric of dai

In [4]:
from langchain_core.output_parsers import StrOutputParser

# Prompt: return short bio of the celebrity
bio_prompt = PromptTemplate(
    input_variables=["celebrity"],
    template="Return a short 2‑sentence bio of the celebrity {celebrity}. Only output the bio text.",
)

# Chain = Prompt -> LLM -> string
bio_chain = bio_prompt | openai_llm | StrOutputParser()

# 1) invoke: single input
print("=== .invoke ===")
one = bio_chain.invoke({"celebrity": "Tom Cruise"})
print(one, "\n")

# 2) batch: multiple inputs
print("=== .batch ===")
celebs = [
    {"celebrity": "Oprah Winfrey"},
    {"celebrity": "Lionel Messi"},
    {"celebrity": "A. R. Rahman"},
]
many = bio_chain.batch(celebs)
for c, b in zip(celebs, many):
    print(c["celebrity"], "->", b, "\n")

# 3) stream: token/chunk streaming
print("=== .stream ===")
for chunk in bio_chain.stream({"celebrity": "Albert Einstein"}):
    print(chunk, end="", flush=True)
print()

=== .invoke ===
Tom Cruise is an American actor and producer known for his roles in blockbuster films such as Top Gun, Mission: Impossible, and Jerry Maguire. He is also a prominent member of the Church of Scientology. 

=== .batch ===
Oprah Winfrey -> Oprah Winfrey is a media mogul, philanthropist, and former talk show host known for her influential and inspirational work. She is also the founder of the Oprah Winfrey Network (OWN) and has been a powerful advocate for various social causes. 

Lionel Messi -> Lionel Messi is an Argentine professional soccer player who is widely considered one of the greatest footballers of all time. He has won numerous awards and accolades throughout his career, including multiple FIFA Ballon d'Or titles. 

A. R. Rahman -> A. R. Rahman is an Indian composer, singer, and music producer known for his work in the film industry. He has won numerous awards, including two Academy Awards and two Grammy Awards. 

=== .stream ===
Albert Einstein was a German-bor

In [6]:
# Async: use ainvoke for 2–3 celebrities (Jupyter supports top‑level await)

celebs = ["Tom Cruise", "Oprah Winfrey", "Lionel Messi"]

for name in celebs:
    bio = await bio_chain.ainvoke({"celebrity": name})
    print(name, "->", bio, "\n")

Tom Cruise -> Tom Cruise is an American actor and producer known for his roles in blockbuster films such as Top Gun, Mission: Impossible, and Jerry Maguire. He is also a prominent member of the Church of Scientology. 

Oprah Winfrey -> Oprah Winfrey is a media mogul, philanthropist, and former talk show host known for her influential and inspirational work. She is also the founder of the Oprah Winfrey Network (OWN) and has been a powerful advocate for various social causes. 

Lionel Messi -> Lionel Messi is an Argentine professional soccer player widely considered one of the greatest of all time. He has won numerous awards and accolades throughout his career, including multiple FIFA Ballon d'Or titles. 



In [7]:
# Async: use abatch for 2–3 celebrities (Jupyter supports top‑level await)

celebs = [
    {"celebrity": "Tom Cruise"},
    {"celebrity": "Oprah Winfrey"},
    {"celebrity": "Lionel Messi"},
]

bios = await bio_chain.abatch(celebs)

for c, b in zip(celebs, bios):
    print(c["celebrity"], "->", b, "\n")

Tom Cruise -> Tom Cruise is an American actor and producer known for his roles in blockbuster films such as Top Gun, Mission: Impossible, and Jerry Maguire. He is also a prominent member of the Church of Scientology. 

Oprah Winfrey -> Oprah Winfrey is a media mogul, philanthropist, and former talk show host known for her influential and inspirational work. She is one of the most successful and powerful women in the entertainment industry. 

Lionel Messi -> Lionel Messi is an Argentine professional soccer player widely considered one of the greatest of all time. He has won numerous awards and accolades throughout his career, including multiple FIFA Ballon d'Or titles. 

